In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import matplotlib.gridspec as gridspec
from random import sample
import random
from keras.layers import Input
from keras.models import Model
from keras.layers import Input, Activation, Add, Dropout,Reshape, Concatenate,PReLU,Embedding,multiply
from keras.layers.core import Dense,Activation,Flatten
from keras.layers import UpSampling2D, Lambda
from keras.datasets import mnist
from keras.models import Model,Sequential
from keras.layers.convolutional import Conv2D,UpSampling2D,Conv2DTranspose
from keras.layers.pooling import AvgPool2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, Adam, RMSprop
from keras import backend as K
import tensorflow as tf
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from skimage.transform import resize
from keras import backend as K

In [ ]:
class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols,self.channels)
        self.num_classes = 12
        self.latent_dim = 100

        optimizer1 = Adam(0.0002, 0.5)
        optimizer2 = Adam(0.00003, 0.5)
        losses = ['binary_crossentropy']
              
        # Build and compile the discriminator
        img_input = Input(shape=(self.img_rows,self.img_cols,self.channels))
        self.discriminator,valid = self.get_discriminator(img_input)
        self.discriminator.compile(loss=losses,
            optimizer=optimizer2,
            metrics=['accuracy'])      


        # Build the generator
        noise = Input(shape=(100,))

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        self.generator = self.get_generator(noise)
        img = self.generator(noise)
        valid = self.discriminator(img)


        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
       
        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(noise,valid)
        self.combined.compile(loss=losses,
            optimizer=optimizer1,
            metrics=['accuracy'])

    """    
    def SubpixelConv2D(self,name, scale=2):
        def subpixel_shape(input_shape):
            dims = [input_shape[0],
                    None if input_shape[1] is None else input_shape[1] * scale,
                    None if input_shape[2] is None else input_shape[2] * scale,
                    int(input_shape[3] / (scale ** 2))]
            output_shape = tuple(dims)
            return output_shape

        def subpixel(x):
            return tf.depth_to_space(x, scale)

        return Lambda(subpixel, output_shape=subpixel_shape, name=name)
    """
    def get_generator(self,input_layer):
        def residual_block(input):
            x = Conv2D(64, kernel_size=3, strides=1, padding='same')(input)
            x = BatchNormalization(momentum=0.8)(x)
            x = LeakyReLU(0.2)(x)            
            x = Conv2D(64, kernel_size=3, strides=1, padding='same')(x)
            x = BatchNormalization(momentum=0.8)(x)
            #x = Add()([x, input])
            return x

        def upsample(x, number):
            x = Conv2D(256, kernel_size=3, strides=1, padding='same', name='upSampleConv2d_'+str(number))(x)
            x = self.SubpixelConv2D('upSampleSubPixel_'+str(number), 2)(x)
            x = PReLU(shared_axes=[1,2], name='upSamplePReLU_'+str(number))(x)
            return x

        hid = Dense(512 * 8 * 8, activation='relu')(input_layer)    
        x_start = Reshape((8, 8, 512))(hid)
        
       
        gen = Conv2D(256, (3,3), padding='same')(x_start)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        # gen = Activation('ReLU')(gen)

        gen = UpSampling2D(size=(2,2))(gen)
        gen = Conv2D(128, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        # gen = Activation('ReLU')(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)

        gen = UpSampling2D(size=(2,2))(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        gen = UpSampling2D(size=(2,2))(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        #gen = upsample(gen,1)
        #gen = upsample(gen,2)
        #gen = upsample(gen,3)
        gen = Conv2D(3, (3,3), padding='same')(gen)
        gen = Activation('tanh')(gen)
    
        """hid = Dense(512 * 12 * 12, activation='relu')(input_layer)    
        x_start = Reshape((12, 12, 512))(hid)
        
       
        gen = Conv2D(256, (3,3), padding='same')(x_start)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        # gen = Activation('ReLU')(gen)

        gen = UpSampling2D(size=(2,2))(gen)
        gen = Conv2D(128, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        # gen = Activation('ReLU')(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)

        gen = UpSampling2D(size=(2,2))(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        gen = UpSampling2D(size=(2,2))(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        gen = Conv2D(64, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)
        #gen = upsample(gen,1)
        #gen = upsample(gen,2)
        #gen = upsample(gen,3)
        gen = Conv2D(3, (3,3), padding='same')(gen)
        gen = Activation('tanh')(gen)
        """

        """  
        x_start = Conv2D(64, kernel_size=9, strides=1, padding='same')(x_start)
        x_start = PReLU(shared_axes=[1,2])(x_start)
        r = residual_block(x_start)
        #for _ in range(3):
        #    r = residual_block(r)
        x = Conv2D(64, kernel_size=3, strides=1, padding='same')(r)
        x = BatchNormalization(momentum=0.8)(x)
        x = Add()([x, x_start])
        
        
        #gen = UpSampling2D(size=(2,2))(x)
        gen = Conv2D(256, (3,3), padding='same')(x)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen)   
        
        gen = residual_block(gen) 
        gen = upsample(gen, 1)
        #gen = UpSampling2D(size=(2,2))(gen)
        gen = Conv2D(256, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen) 
        
        gen = upsample(gen, 2)
        gen = residual_block(gen) 
        gen = residual_block(gen) 
        gen = upsample(gen, 3)
        
        #gen = UpSampling2D(size=(2,2))(gen)
        gen = Conv2D(256, (3,3), padding='same')(gen)
        gen = BatchNormalization(momentum = 0.5)(gen)
        gen = LeakyReLU(0.2)(gen) 
        
        gen = residual_block(gen) 
        gen = residual_block(gen) 
        """

        #x = upsample(x, 1)
        #x = upsample(x, 2)
        #x = upsample(x, 3)
        hr_output = Conv2D(
                3, 
                kernel_size=9, 
                strides=1, 
                padding='same', 
                activation='tanh'
            )(gen)
        model = Model(inputs=input_layer, outputs=hr_output)  
        model.summary()
        return model
    

    def get_discriminator(self,input_layer):
            def conv2d_block(input, filters, strides=1, bn=True):
                d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(input)
                d = LeakyReLU(alpha=0.2)(d)
                if bn:
                    d = BatchNormalization(momentum=0.8)(d)
                return d
            def residual_block(input,filters=64):
                x = Conv2D(filters, kernel_size=3, strides=1, padding='same')(input)
                x = BatchNormalization(momentum=0.8)(x)
                x = PReLU(shared_axes=[1,2])(x)            
                x = Conv2D(filters, kernel_size=3, strides=1, padding='same')(x)
                x = BatchNormalization(momentum=0.8)(x)
                x = Add()([x, input])
                return x

            filters = 32
            x = Conv2D(filters, kernel_size=3, strides=2, padding='same')(input_layer)
            x = LeakyReLU(alpha=0.2)(x)
            x = residual_block(x,filters)
            #x = residual_block(x,filters)
            x = Conv2D(filters*4, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = residual_block(x, filters*4)
            x = residual_block(x, filters*4)
            x = Conv2D(filters*8, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = residual_block(x, filters*8)
            x = residual_block(x, filters*8)
            x = Conv2D(filters*8, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = residual_block(x, filters*8)
            x = residual_block(x, filters*8)
            x = Conv2D(filters*8, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            """            
            x = Conv2D(filters, kernel_size=3, strides=2, padding='same')(input_layer)
            x = LeakyReLU(alpha=0.2)(x)
            x = residual_block(x,filters)
            #x = residual_block(x,filters)
            x = Conv2D(filters*2, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = residual_block(x, filters*2)
            x = residual_block(x, filters*2)
            x = Conv2D(filters*4, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = residual_block(x, filters*4)
            x = residual_block(x, filters*4)
            x = Conv2D(filters*8, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            #x = residual_block(x, filters*8)
            x = residual_block(x, filters*8)
            x = Conv2D(filters*16, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            #x = residual_block(x, filters*8)
            x = residual_block(x, filters*16)
            x = Conv2D(filters*32, kernel_size=3, strides=2, padding='same')(x)
            x = LeakyReLU(alpha=0.2)(x)
            """
            x = Flatten()(x)
            x1 = Dense(1, activation='sigmoid')(x)
            
            model = Model(inputs=input_layer, outputs=x1)
            model.summary()
            return model,x1
    def train(self,X_train, epochs, batch_size=128, sample_interval=50,img_name=''):

        # Load the dataset
        #(X_train, y_train), (_, _) = mnist.load_data()

        # Configure inputs
        #X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        #X_train = np.expand_dims(X_train, axis=3)
        #y_train = y_train.reshape(-1, 1)
        #print('y_train',y_train)
        
        # Adversarial ground truths
        #self.generator.load_weights('./No_Res_Module/generator_weights.hdf5',by_name=True)
        #self.discriminator.load_weights('./No_Res_Module/discriminator_weights.hdf5',by_name=True)
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        pre_g_loss = 0
        id_list = [_ for _ in range(X_train.shape[0])]
        random.shuffle(id_list)
        
        for epoch in range(epochs):
            for mini_batch in range(X_train.shape[0]//batch_size-2):
                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                #idx = np.random.randint(0, X_train.shape[0], batch_size)
                idx = np.array([id_list[_] for _ in range(batch_size*mini_batch,batch_size*(mini_batch+1))])
                imgs = X_train[idx]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))
                #print('imgs',np.shape(imgs))
                # The labels of the digits that the generator tries to create an
                # image representation of
                #sampled_labels = np.random.randint(0, self.num_classes, (batch_size, 1))
                # Generate a half batch of new images
                gen_imgs = self.generator.predict(noise)
                #print('G',np.shape(gen_imgs))
                # Image labels. 0-11 if image is valid or 12 if it is generated (fake)
                #img_labels = y_train[idx]
                #fake_labels = self.num_classes * np.ones(img_labels.shape)
                #print('D',(self.discriminator.predict(gen_imgs)))
                # Train the discriminator
                if mini_batch % 3 == 0 and pre_g_loss < 5:
                    d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                    #print('real_loss',d_loss_real)
                    d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator
                g_loss = self.combined.train_on_batch(noise, valid)
                pre_g_loss = g_loss[0]
                # Plot the progress
                #print('d_loss',d_loss)
                #print('g_loss',g_loss)
                print ("Epoch%d,MiniBatch:%d [D loss: %f, acc.: %.2f%%] [G loss: %f，G acc: %f]" % (epoch,mini_batch, d_loss[0], 100*d_loss[1], g_loss[0],g_loss[1]))

                # If at save interval => save generated image samples
                if mini_batch % sample_interval == 1 or mini_batch == (X_train.shape[0]//batch_size-1):
                    self.save_model()
                    self.sample_images(epoch,mini_batch,img_name)
            

    def sample_images(self, epoch,mini_batch,img_name):
        r, c = 2, 6
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([_ for _ in range(r*c)])
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        gen_imgs = self.combine_images(gen_imgs) 
        """        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gen_imgs/%d_%d.png" % (epoch,mini_batch))
        plt.close()
        """
          
        plt.imshow(gen_imgs)
        plt.axis('off')
        plt.savefig("gen_imgs/%s%d_%d.png" % (img_name,epoch,mini_batch))
        plt.close()
        
        
    
    def combine_images(self, generated_images):
        num = generated_images.shape[0]
        width = 6
        height = 2
        shape = generated_images.shape[1:3]
        image = np.zeros((height*shape[0], width*shape[1],3),
                         dtype=generated_images.dtype)
        for index, img in enumerate(generated_images):
            i = int(index/width)
            j = index % width
            image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1],:] = img[:, :, :]
        return image
    
    def save_model(self):
        def save(model, model_name):
            model_path = "No_Res_Module/%s.json" % model_name
            weights_path = "No_Res_Module/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator_extra")
        save(self.discriminator, "discriminator_extra")

In [ ]:
tags = pd.read_csv('./extra_data/extra_data/convert_tags.csv')
X_train = np.array([(np.array(load_img("./extra_data/extra_data/images/{}.jpg".format(ids))) / 127.5 - 1) for ids in tqdm_notebook(tags.idx)])
np.shape(X_train)

In [ ]:
X_train = np.array([np.fliplr(x) for x in X_train])

In [ ]:
np.shape(X_train)

In [ ]:
acgan = ACGAN()

In [ ]:
acgan.train(X_train,epochs=20, batch_size=32, sample_interval=200,img_name='extra_')


In [ ]:
K.clear_session()
X_train = np.array([np.fliplr(x) for x in X_train])
acgan.train(X_train,epochs=20, batch_size=32, sample_interval=200,img_name='flip')